In [6]:
import numpy as np
import pandas as pd

import torch

In [4]:
irisDF = pd.read_csv("../data/iris.csv")
irisDF.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa


In [29]:
X = irisDF.iloc[:, :-1]
Y = irisDF.iloc[:, -1]

In [30]:
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

# OneHotEncoder(sparse_output=True)

In [31]:
# Y = pd.DataFrame(irisDF.iloc[:, -1])
# OH = OneHotEncoder(sparse_output=False)
# onehot_y = OH.fit_transform(Y)
# onehot_y

In [32]:
Label = LabelEncoder()
Y = Label.fit_transform(Y)
Y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [44]:
# 사용자 정의 DataSet Class
class irisDataset(Dataset): # torch.utils.data.Dataset을 부모로 두고 있음
    # 초기화 함수
    def __init__(self, x_data, y_data):
        super().__init__() # 부모 클래스 
        
        # x 데이터 => ndarray
        if isinstance(x_data, torch.Tensor):
            # 이미 텐서라면 형변환
            self.feature =  x_data.type(torch.FloatTensor)
        else:
            x_data = x_data.values if isinstance(x_data, pd.DataFrame) else x_data # 데이터 프레임이라면 valuse로 어레이 뽑아줘야함 
            self.feature = torch.FloatTensor(x_data)
            
        # y 데이터 => ndarray
        if isinstance(y_data, torch.Tensor):
            self.target = y_data.type(torch.LongTensor)
        else:
            y_data = y_data.values if isinstance(y_data, pd.DataFrame) else y_data 
            self.target = torch.LongTensor(y_data) 
        
        # 부가적인 것들(필수 X)
        self.classes = torch.unique(self.target).tolist()
        self.nclasses = len(self.classes)
        self.len = len(self.feature)
        
    # 데이터셋의 개수 체크 함수
    def __len__(self):
        return self.target.shape[0]
    
    # 특정 인덱스 데이터 + 라벨 반환 callback function 
    def __getitem__(self, index):
        return self.feature[index], self.target[index] # 둘 다 튜플로 출동 

In [52]:
irisDS = irisDataset(X, Y)

In [53]:
# 강사님 코드
class CustomDataset(Dataset):
    def __init__(self, x_data, y_data) -> None:
        super().__init__()
        self.x = torch.FloatTensor(x_data)
        self.y = torch.LongTensor(y_data)
        self.classes = np.unique(self.y).tolist()
        self.nclasses = len(self.classes)
        self.len = len(self.x_data)
        
    def __len__(self):
        return self.len
    
    def __getitem__(self, index) -> tuple:
        return self.x[index], self.y[index]

In [63]:
print(f"irisDF.y => {irisDS.feature.dtype}, {irisDS.feature.shape}, {irisDS.feature.ndim}") # dtype, shape, ndim float32, 154, 2
print(f"irisDF.y => {irisDS.target.dtype}, {irisDS.target.shape}, {irisDS.target.ndim}") # int64, 150, 1
print(f"irisDF.len => {irisDS.len}")
print(f"irisDF.classes => {irisDS.classes}")

irisDF.y => torch.float32, torch.Size([150, 4]), 2
irisDF.y => torch.int64, torch.Size([150]), 1
irisDF.len => 150
irisDF.classes => [0, 1, 2]


# DataLoader
- 에포크 : 처음부터 끝까지 학습하는 횟수
- 배치크기 : 전체 데이터를 작은 단위로 나눈 크기 2의 제곱수 크기
- 이터레이션 : 에포크, 배치크기로 계산한 반복 횟수 W, b 업데이트 횟수
- Dataset에서 데이터가 나오는 것이 아닌 __인덱스__ 가 나옴(샘플러)
- DataLoader(dataset, batch_size, shuffle, sampler, batch_sampler, num_worker...

In [76]:
# 로딩된 데이터 확인 함수
def print_batch_data(loader, epochs, batch_size=1, shuffle=False, drop_last=False, sampler=None) -> None:
    print(f"[설정값] Batch size: {batch_size}, shuffle: {shuffle}, drop_last: {drop_last}, sampler: {sampler}")  
    
    # print(f"[loader] {loader}")
    
    for ep in range(epochs):
        print(f"[{ep} EPOCH] ===== batch : {len(loader)}개")
        for (feature, label) in loader:
            print(feature.shape, label.shape, label)

In [77]:
# [기본사용] batch_size=1, suffle=False, drop_last=False, sampler=None

loader = DataLoader(irisDS)
print_batch_data(loader=loader, epochs=2)

[설정값] Batch size: 1, shuffle: False, drop_last: False, sampler: None
[0 EPOCH] ===== batch : 150개
torch.Size([1, 4]) torch.Size([1]) tensor([0])
torch.Size([1, 4]) torch.Size([1]) tensor([0])
torch.Size([1, 4]) torch.Size([1]) tensor([0])
torch.Size([1, 4]) torch.Size([1]) tensor([0])
torch.Size([1, 4]) torch.Size([1]) tensor([0])
torch.Size([1, 4]) torch.Size([1]) tensor([0])
torch.Size([1, 4]) torch.Size([1]) tensor([0])
torch.Size([1, 4]) torch.Size([1]) tensor([0])
torch.Size([1, 4]) torch.Size([1]) tensor([0])
torch.Size([1, 4]) torch.Size([1]) tensor([0])
torch.Size([1, 4]) torch.Size([1]) tensor([0])
torch.Size([1, 4]) torch.Size([1]) tensor([0])
torch.Size([1, 4]) torch.Size([1]) tensor([0])
torch.Size([1, 4]) torch.Size([1]) tensor([0])
torch.Size([1, 4]) torch.Size([1]) tensor([0])
torch.Size([1, 4]) torch.Size([1]) tensor([0])
torch.Size([1, 4]) torch.Size([1]) tensor([0])
torch.Size([1, 4]) torch.Size([1]) tensor([0])
torch.Size([1, 4]) torch.Size([1]) tensor([0])
torch.Siz

In [78]:
# batch_size = 22로 설정

loader = DataLoader(irisDS, batch_size=22)
print_batch_data(loader=loader, epochs=2, batch_size=22)
# 마지막은 모자랍

[설정값] Batch size: 22, shuffle: False, drop_last: False, sampler: None
[0 EPOCH] ===== batch : 7개
torch.Size([22, 4]) torch.Size([22]) tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
torch.Size([22, 4]) torch.Size([22]) tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
torch.Size([22, 4]) torch.Size([22]) tensor([0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
torch.Size([22, 4]) torch.Size([22]) tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
torch.Size([22, 4]) torch.Size([22]) tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
torch.Size([22, 4]) torch.Size([22]) tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
torch.Size([18, 4]) torch.Size([18]) tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
[1 EPOCH] ===== batch : 7개
torch.Size([22, 4]) torch.Size([22]) tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [79]:
# batch_size = 22로 설정 drop_last = True, shuffle = True

loader = DataLoader(irisDS, shuffle=True, batch_size=22, drop_last=True)
print_batch_data(loader=loader, epochs=2, batch_size=22, drop_last=True, shuffle=True) # 셔플은 에포크 단위로 
# 위에서는 모자란 상태로 진행했다면 지금은 그냥 날림 -> 6개 됨

[설정값] Batch size: 22, shuffle: True, drop_last: True, sampler: None
[0 EPOCH] ===== batch : 6개
torch.Size([22, 4]) torch.Size([22]) tensor([0, 1, 1, 1, 2, 0, 2, 0, 1, 1, 0, 2, 2, 0, 1, 0, 2, 0, 2, 1, 0, 2])
torch.Size([22, 4]) torch.Size([22]) tensor([2, 0, 2, 2, 2, 2, 1, 2, 2, 0, 0, 0, 0, 1, 0, 2, 2, 2, 0, 0, 1, 1])
torch.Size([22, 4]) torch.Size([22]) tensor([1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 2, 1, 1, 1, 0, 2, 1, 0, 2])
torch.Size([22, 4]) torch.Size([22]) tensor([2, 1, 1, 1, 1, 0, 1, 2, 2, 2, 1, 1, 2, 0, 0, 0, 0, 2, 1, 1, 2, 1])
torch.Size([22, 4]) torch.Size([22]) tensor([0, 2, 0, 1, 1, 0, 0, 0, 1, 0, 0, 2, 2, 2, 2, 0, 2, 2, 0, 2, 1, 2])
torch.Size([22, 4]) torch.Size([22]) tensor([1, 2, 2, 0, 1, 0, 0, 0, 1, 1, 2, 2, 0, 1, 1, 1, 1, 2, 2, 2, 2, 0])
[1 EPOCH] ===== batch : 6개
torch.Size([22, 4]) torch.Size([22]) tensor([0, 1, 1, 2, 1, 2, 1, 2, 2, 2, 0, 1, 2, 1, 0, 2, 2, 1, 1, 2, 1, 0])
torch.Size([22, 4]) torch.Size([22]) tensor([2, 1, 2, 2, 1, 0, 2, 2, 0, 1, 1, 0, 1, 2, 0, 1, 0

In [79]:
# https://pytorch.org/docs/stable/_modules/torch/utils/data/sampler.html#Sampler

In [ ]:
# class WeightedRandomSampler